# Script de préparation des données en vue de l'agrégation de Slave Voyages et de Maritime History

### Imports
* Python fonctionne avec un noyau de base et des "librairies", qui sont des séries de fonctions spécialisées chacune dans un domaine. Il faut déclarer celles dont on a besoin en début de script pour qu'elles soient actives.
* Ici, la principale à noter est [Pandas](pandas.pydata.org/) qui donne une grande efficacité dans le traitement des données tabulaires.

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import csv

* Après avoir activé nos librairies, nous allons déclarer nos fonctions locales, c'est-à-dire des suites d'instructions 'customisées' que j'ai écrites pour cet atelier. Les déclarer en amont permet de simplifier l'écriture et la lecture du script.

### FONCTION : faire un DataFrame Pandas à partir des résultats¶

In [ ]:
def query_to_df(spql_queried):
    
    preparing = {}
    
    try:
        spql_return = spql_queried.queryAndConvert()
        
        for ret in spql_return["results"]["bindings"]:
            for var in ret.keys():
                if var not in preparing.keys():
                    preparing[var] = []
                    
        for ret in spql_return["results"]["bindings"]:
            for var in preparing.keys():
                if var in ret.keys():
                    preparing[var].append(ret[var]['value'])
                else:
                    preparing[var].append('None')
        return pd.DataFrame(preparing)

    except Exception as e:
        print("The query has a problem. Here is the error:\n\t", e)

### FONCTION : forcer un datetime type sur une colonne

In [ ]:
def force_datetime(df, column, date_format):

    new_column = []
    for cell in df[column]:
        try:
            if cell == '' or cell == None:
                new_column.append(cell)
            else:
                date_time_obj = datetime.strptime(cell, date_format)
                new_column.append(date_time_obj)
        except Exception as e:
            print(f'La cellule "{cell}" dans la colonne "{column}" a généré l\'erreur suivante : "{e}" et a été gardée dans son état originel.')
            new_column.append(cell)

    return new_column

# Importons nos deux bases

### IMPORT et exploration de Slave Voyages
* Import des données.

In [ ]:
slave_voyages_orig = pd.read_csv(
    "../../data/slave-voyages_trans-atlantic-db.csv",
    skip_blank_lines=False,
    infer_datetime_format = True,
    na_filter=False
)

* Restriction des colonnes. (Le tableau de Slave Voyages contient de nombreuses colonnes, mais nous n'en utiliserons que quatre ici, pas la peine donc de surcharger la mémoire.)

In [ ]:
slave_voyages = slave_voyages_orig[["Flag of vessel", 'Date vessel departed with captives', 'Vessel name', "Captain's name"]]

* Voyons donc la tête de notre DataFrame.

In [ ]:
slave_voyages.head(5)

### IMPORT et exploration de Maritime History
* C'est un peu plus complexe ici. Nous travaillons avec des données liées (format RDF), il faut donc interroger la base avec le langage SPARQL, et nous en tirons un tableau grâce à la fonction `query_to_df` définie en haut.
* Pour référence, je l'exporte en CSV avant travail.
* Et on regarde de quoi le DataFrame de travail a l'air.

In [ ]:
mh_endpoint = SPARQLWrapper("https://sparql.geovistory.org/api_v1_project_84760")
mh_endpoint.setReturnFormat(JSON)
mh_endpoint.setQuery("""
    PREFIX onto: <http://www.ontotext.com/>
    PREFIX ont: <http://purl.org/net/ns/ontology-annot#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xml: <http://www.w3.org/XML/1998/namespace>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX time: <http://www.w3.org/2006/time#>
    PREFIX ontome: <https://ontome.net/ontology/>
    PREFIX geov: <http://geovistory.org/resource/>
    
    SELECT ?voyage ?voyage_label ?voyage_start ?voyage_end ?ship ?ship_name ?participant ?participant_label
    WHERE {
    ?voyage rdf:type ontome:c523 ;
        rdfs:label ?voyage_label ;
        ontome:p4 ?tspan ;
        ontome:p1338 ?ship .
  ?ship rdfs:label ?ship_name .
  ?tspan ontome:p151/rdfs:label ?voyage_end ;
    ontome:p150/rdfs:label ?voyage_start .
  OPTIONAL {
    ?voyage ontome:p1359 ?participant .
    ?participant ontome:p1111i/rdfs:label ?participant_label .
  }
}
""")

maritime_history = query_to_df(mh_endpoint)
maritime_history.to_csv("output_voyages.csv")
maritime_history.head(5)

### Quels sont les types de données ?
* Le typage est essentiel pour la bonne marche du nettoyage. Voyons ce que notre import a fait.
* D'abord, sur Maritime History :

In [ ]:
print(slave_voyages.info())

* Puis sur Slave Voyages :

In [ ]:
print(maritime_history.info())

### Aperçu des types de colonnes pour Slave Voyages

In [ ]:
for column in list(slave_voyages):
    print(f'Colonne "{column}", 14e cellule:\n\t\t- type "{type(slave_voyages[column][13])}"\n\t\t- (valeur : "{slave_voyages[column][13]}")\n')

### Aperçu des types de colonnes pour Maritime History

In [ ]:
for column in list(maritime_history):
    print(f'Colonne "{column}", 14e cellule:\n\t\t- type "{type(maritime_history[column][13])}"\n\t\t- (valeur : "{maritime_history[column][13]}")\n')

### Séparer les lignes avec plusieurs noms de capitaines
* D'abord, quelques informations.

In [ ]:
column = "Captain's name"
print(f"Il y a {len(np.unique(slave_voyages[column]))} noms différents dans Slave Voyages.")
print(f"Il y a {len(np.unique(maritime_history['participant_label']))} noms différents dans Maritime History.")

* Voyons à quoi ressemblent les noms de capitaines de chaque base.

In [ ]:
print("Exemple de nom dans Slave Voyages :\t" + np.unique(slave_voyages["Captain's name"])[5])
print("Exemple de nom dans Maritime History :\t" + np.unique(maritime_history["participant_label"])[5])

* Il faut donc changer un système de notation en un autre. J'ai choisi de changer Slave Voyages : repérer les lignes avec plusieurs capitaines différents, et en faire une ligne par capitaine (autres champs copiés exactement).

In [ ]:
print(f"{len(slave_voyages)} lignes avant traitement.")

dropem = []
for index, row in slave_voyages.iterrows():
    if '<br' in row["Captain's name"]:
        for item in row["Captain's name"].split("<br/>"):
            new_row = {
                'Flag of vessel' : row['Flag of vessel'],
                'Date vessel departed with captives' : row['Date vessel departed with captives'],
                'Vessel name' : row['Vessel name'],
                "Captain's name" : item.lstrip().rstrip()
            }
            slave_voyages = slave_voyages.append(new_row, ignore_index = True)
        dropem.append(index)

slave_voyages.drop(dropem, inplace=True)
slave_voyages.reset_index(drop=True, inplace=True)

print(f"{len(slave_voyages)} lignes après traitement.")

### Séparer les noms corrigés des graphies originelles

In [ ]:
alt_spelling = []
captains = []


for captain in slave_voyages['Captain\'s name']:
    
    if '(a)' in captain:
        alt_spelling.append(captain.split('(a)')[1].lstrip().rstrip())
        captains.append(captain.split('(a)')[0].lstrip().rstrip())

    else:
        alt_spelling.append('')
        captains.append(captain)

slave_voyages["Original Captain's name"] = alt_spelling
slave_voyages["Corrected Captain's name"] = captains

slave_voyages.tail(5)

### Réordonner les noms des capitaines

In [ ]:
captains = []

for captain in slave_voyages['Corrected Captain\'s name']:
    
    if ',' in captain:
        split_n = captain.split(',')
        captains.append(split_n[1].lstrip().rstrip() + ' ' + split_n[0].lstrip().rstrip())
    else:
        captains.append(captain)

slave_voyages["Reordered corrected Captain's name"] = captains

slave_voyages.tail(5)

### Séparer les noms de navires corrigés des graphies originelles

In [ ]:
alt_spelling = []
ships = []


for ship in slave_voyages['Vessel name']:
    
    if '(a)' in ship:
        alt_spelling.append(ship.split('(a)')[1].lstrip().rstrip())
        ships.append(ship.split('(a)')[0].lstrip().rstrip())

    else:
        alt_spelling.append('')
        ships.append(ship)

slave_voyages["Original Vessel name"] = alt_spelling
slave_voyages["Corrected Vessel name"] = ships

slave_voyages.head(5)

### Réordonner les noms des navires

In [ ]:
ships = []

for ship in slave_voyages['Corrected Vessel name']:
    
    if "'s-" in ship:
        split_n = ship.split('-')
        ships.append(split_n[1].lstrip().rstrip() + ' ' + split_n[0].lstrip().rstrip())
    else:
        ships.append(ship)

slave_voyages["Reordered corrected Vessel name"] = ships

slave_voyages.tail(5)

### Typage des dates
* Que trouve-t-on dans Slave Voyages ?

In [ ]:
print(np.unique(slave_voyages['Date vessel departed with captives'])[5])
print(type(np.unique(slave_voyages['Date vessel departed with captives'])[5]))

* Que trouve-t-on dans Maritime History ?

In [ ]:
print(np.unique(maritime_history['voyage_start'])[5])
print(type(np.unique(maritime_history['voyage_start'])[5]))

* La date minimum gérée par le type DateTime64 de Pandas (son seul point faible), c'est :

In [ ]:
pd.Timestamp.min

* Dommage qu'en histoire, on ait très souvent besoin de dates antérieures. Il va falloir faire autrement :
* On ne peut peut-être pas typer la colonne comme contenant des dates, mais on peut typer chaque objet comme étant une date. Cela nécessite un peu de temps à vérifier quel est le format utilisé pour le préciser, et une petite fonction _ad hoc_ écrite par @gaetanmuck et incluse en haut.

In [ ]:
maritime_history['voyage_start'] = force_datetime(maritime_history, "voyage_start", '%Y-%m-%d')
maritime_history['voyage_end'] = force_datetime(maritime_history, "voyage_end", '%Y-%m-%d')
slave_voyages['Date vessel departed with captives'] = force_datetime(slave_voyages, "Date vessel departed with captives", '%Y-%m-%dT%H:%M:%SZ')